# Recursive Clustering and Summarization

Plan:
- recursively cluster collections
- create tree of clusters (the HDBSCAN does this anyways but likely not as we want)
- cluster until max-depth is reached or (better) until each leaf only has one "plausible" cluster (based on thresholds or probabilities)
- try summarizing to get "main idea" out of cluster


- cluster on keywords ( randomize all grammar + stop words )
- topic clusters
- context: title, abstract, etc. keywords



## Recursively cluster 

Based on the topic_clustering notebook, we will try with Agglomerative Clustering 

In [4]:
import pandas as pd

df = pd.read_csv("downloads/40k_balanced_pm_acl.csv")#.sample(frac=0.5)

In [5]:
!python --version

Python 3.6.13


In [6]:
pos = df[df.labels == 1]


sentences = list(pos["text"]) #otherwise key error

In [4]:
#pos_h = pos[pos["source"].isin(["Oct1_clinical_studies_pm",'oct3_labels', 'labels_oct7'])]

In [5]:
#sents_h = list(pos_h["text"]) #otherwise key error

In [1]:
from sentence_transformers import SentenceTransformer, util

print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus ... get a coffee in the meantime


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

NameError: name 'sentences' is not defined

In [ ]:
len(embeddings)

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np

def cluster(embeddings, **kwargs):
    embeddings = embeddings.cpu()
    # Normalize the embeddings to unit length
    corpus_embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

    # Perform kmean clustering
    clustering_model = AgglomerativeClustering(**kwargs) #, affinity='cosine', linkage='average', distance_threshold=0.4)
    clustering_model.fit(corpus_embeddings)
   # cluster_assignment = clustering_model.labels_
    return clustering_model

In [ ]:
def get_clusters(clustering_model):
    
    clusters = {}
    for sentence_id, cluster_id in enumerate(clustering_model.labels_):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        try:
            clusters[cluster_id].append(sentences[sentence_id])
        except:
            print(sentence_id, "sentence_id")
    return clusters
   
#     for i, cluster in clustered_sentences.items():
#         print("Cluster ", i+1)
#         print(cluster)
#         print("\n")

In [12]:
sample = embeddings

In [13]:
cluster_model = cluster(sample, n_clusters=None, distance_threshold=1.4)

Cluster Model attributes

   n_clusters_ : int
        The number of clusters found by the algorithm. If
        ``distance_threshold=None``, it will be equal to the given
        ``n_clusters``.

    labels_
    n_leaves_

    n_connected_components_ : The estimated number of connected components in the graph.

    children_ : array-like of shape (n_samples-1, 2)
        The children of each non-leaf node. Values less than `n_samples`
        correspond to leaves of the tree which are the original samples.
        A node `i` greater than or equal to `n_samples` is a non-leaf
        node and has children `children_[i - n_samples]`. Alternatively
        at the i-th iteration, children[i][0] and children[i][1]
        are merged to form node `n_samples + i`

    distances_ : array-like of shape (n_nodes-1,)
        Distances between nodes in the corresponding place in `children_`.
        Only computed if `distance_threshold` is used or `compute_distances`
        is set to `True`.

In [ ]:
# import itertools

# ii = itertools.count(sample.shape[0])
# [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in cluster_model.children_]

In [ ]:
len(get_clusters(cluster_model).keys())

In [ ]:
clusters = get_clusters(cluster_model)

In [ ]:
import matplotlib.pyplot as plt

for thresh in [1, 1.2, 1.4, 1.8]:
    cluster_model = cluster(sample, n_clusters=None, distance_threshold=thresh)
    plt.hist([len(v) for v in get_clusters(cluster_model).values()], bins='auto', label=str(thresh))
    plt.title("Threshold " + str(thresh))

In [ ]:
def collection_to_clusters(texts, model=model,  **kwargs):
    embs = model.encode(texts, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
    cluster_model = cluster(embs, **kwargs)
    return get_clusters(cluster_model).values()

In [ ]:
cluster_tree = {}
lens = [len(v) for v in clusters.values()]
for i, v in clusters.items():
    if len(v) > 25:
        cluster_tree[i] = {"parent" : v}
        cluster_tree[i] = {"children" :[*collection_to_clusters(v, n_clusters=None, distance_threshold=0.4)]}
        #get values, embed and sample again with lower threshold
        

In [ ]:
for i,v in cluster_tree.items():
    print(i)
    for sent in v["children"]:
        print(sent)
        print("\n\n")
    print("-------------- \n\n")

In [ ]:
#dict(enumerate(cluster_model.children_, cluster_model.n_leaves_))

## Summarization

**Tried: Google Pegasus**. Result: Does a terrible job of keeping the important information and doesn't retain the question but guesses at a conclusion


In [ ]:
torch.cuda.is_available()

### Pegasus Setup


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)


In [ ]:

def summarize(sentences):
    batch = tokenizer(sentences, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

### T5 Setup

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ Background: Trust is a critical component of competency committees given their high-stakes decisions. Research from outside of medicine on group trust has not focused on trust in group decisions, and "group trust" has not been clearly defined. The purpose was twofold: to examine the definition of trust in the context of group decisions and to explore what factors may influence trust from the perspective of those who rely on competency committees through a proposed group trust model. Methods: The authors conducted a literature search of four online databases, seeking articles published on trust in group settings. Reviewers extracted, coded, and analyzed key data including definitions of trust and factors pertaining to group trust. Results: The authors selected 42 articles for full text review. Although reviewers found multiple general definitions of trust, they were unable to find a clear definition of group trust and propose the following: a group-directed willingness to accept vulnerability to actions of the members based on the expectation that members will perform a particular action important to the group, encompassing social exchange, collective perceptions, and interpersonal trust. Additionally, the authors propose a model encompassing individual level factors (trustor and trustee), interpersonal interactions, group level factors (structure and processes), and environmental factors."""
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("snrspeaks/t5-one-line-summary") #snrspeaks/t5-one-line-summary
tokenizer = AutoTokenizer.from_pretrained("snrspeaks/t5-one-line-summary")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
tokenizer.decode(outputs[0])
def summarize(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0])

In [ ]:
summarize("I have seen a ghost in my shed")

In [ ]:
"I have seen a ghost in my shed"[:5]

### Print Results

In [ ]:
for ID, cluster in get_clusters(cluster_model).items():
    sentences = ".".join(cluster)
    print(sentences)
    print( "\n\n", "sum:::", summarize(sentences[:256]), "\n\n\n")
    

In [ ]:
!pip bertopic --version

In [ ]:
!pip uninstall bertopic --yes

In [ ]:
pip uninstall umap
pip install umap-learn

In [5]:
!pip uninstall bertopic

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.


In [7]:
!pip uninstall numpy --yes

Found existing installation: numpy 1.19.5
Uninstalling numpy-1.19.5:
  Successfully uninstalled numpy-1.19.5


## BERTopic topic modeling + set intersection

In [7]:
from bertopic import BERTopic
m = BERTopic(embedding_model='sentence-transformers/all-mpnet-base-v2', calculate_probabilities=True, verbose=True)

In [ ]:
topics, probs = m.fit_transform(sentences)

In [ ]:
m.get_topic(0)  # Select the most frequent topic

In [ ]:
m.get_topic(1)  # Select the most frequent topic

In [ ]:
#samp = sentences[1100]

def extract_topics(texts, distance_thresh=0.035, model=m):
    topic_labels = model.transform(texts)[0]
    topic_dict = {}
    for idx, ID in enumerate(topic_labels):
        topics = model.get_topic(ID)
        for t in topics:
            topic = t[0]
            dist = t[1]
            if dist > distance_thresh:
                if not topic_dict.get(topic):
                    topic_dict[topic] = [ texts[idx] ] 
                else:
                    sents = topic_dict[topic] + [texts[idx]]
                    topic_dict[topic] = sents
    return topic_dict

In [ ]:
td = extract_topics(sentences)

In [62]:
td

{'nursing': ['The difficulty with this task lies in the fact that prosodic cues are never absolute ; they are relative to individual speakers , gender , dialect , discourse context , local context , phonological environment , and many other factors',
  'The second problem of traditional word alignment approaches is the fact that parameter estimations are usually based on plain text items only',
  'To overcome the disadvantage of measure word generation in a general SMT system',
  'Perhaps the biggest limitation is the small number of research participants',
  'To cope with the problem of sparse training resources',
  'We also address the issue of sparse or impoverished training data',
  'Although there exist abundant collections of raw text , the high expense of manually annotating the text',
  'The primary weakness of this approach is that it relies too heavily on user interaction',
  'Another limitation that may have affected the accuracy of the model was the way in which potential p

In [63]:
import json

with open("full_topic_triples.json", "w") as f:
    json.dump(td, f)

## Hierachy by source text overlap


In [22]:
#take td
#if there's overlap of t1 and t2 t1 is parent if t2 has less of the sources (and 80%+) of t2 sources are in t1 (otherwise duplicate or not that realted))

In [73]:


def sources_relation(s1, s2, min_overlap=0.7):
    s1 = set(s1)
    s2 = set(s2)
    l1 = len(s1)
    l2 = len(s2)

    common = s1.intersection(s2) #order doesn't matter
    
#     cl1 = len(s1.intersect(common))
#     cl2 = len(s2.intersect(common))
    
    relation="TBA"
    #if only a few sources are common and it's not all/most of one topics sources then -> different ideas
    #if many are common and one topic is much bigger, the smaller is a subtopic
    #if almost all are common and there's 50:50 or 30:70 split then they are duplicates
    #either unrelated, duplicate, or hypernomy
    
    cl = len(common)

    ol2 = cl/l2
    ol1 = cl/l1
    if l1 > l2:
        ol = ol2
    else:
        ol = ol1
    
    if ol < min_overlap: #little in common (one topic is enough to decide)
        relation="different"
        
        #-- comparative
    if ol > min_overlap: #if lots of overlap
        if l2/l1 < 8/10:
            relation="child"
            
        elif l1/l2 < 8/10:
            relation="parent"
        else: #approx similar size
            #topics similar size and much of overlap (synonyms)
            relation = "duplicates"
    
    return relation


In [74]:
from itertools import combinations

topic_pairs = combinations(td.keys(), 2)
result = [(t1, t2, sources_relation(td[t1], td[t2]) ) for t1,t2 in topic_pairs]


In [75]:
result

[('nursing', 'students', 'duplicates'),
 ('nursing', 'stroke', 'different'),
 ('nursing', 'disability', 'different'),
 ('nursing', 'parkinson', 'different'),
 ('nursing', 'pd', 'different'),
 ('nursing', 'hemiplegia', 'different'),
 ('nursing', 'thrombectomy', 'different'),
 ('nursing', 'endovascular', 'different'),
 ('nursing', 'mechanical', 'different'),
 ('nursing', 'health', 'different'),
 ('nursing', 'eating', 'different'),
 ('nursing', 'children', 'different'),
 ('nursing', 'diabetes', 'different'),
 ('nursing', 'overweight', 'different'),
 ('nursing', 'medication', 'different'),
 ('nursing', 'obesity', 'different'),
 ('nursing', 'adherence', 'different'),
 ('nursing', 'depression', 'different'),
 ('nursing', 'diabetic', 'different'),
 ('nursing', 'disease', 'different'),
 ('nursing', 'cardiovascular', 'different'),
 ('nursing', 'myocardial', 'different'),
 ('nursing', 'femoral', 'different'),
 ('nursing', 'osteoarthritis', 'different'),
 ('nursing', 'musculoskeletal', 'different

### Building the Tree

In [67]:
triples = [r for r in result if r[2] == "child" or r[2] == "parent"]

In [72]:
triples[:8], len(result), len(triples)

([('health', 'eating', 'child'),
  ('health', 'children', 'child'),
  ('health', 'overweight', 'child'),
  ('health', 'medication', 'child'),
  ('health', 'obesity', 'child'),
  ('health', 'adherence', 'child'),
  ('health', 'depression', 'child'),
  ('eating', 'diabetes', 'parent')],
 1225,
 29)

In [2]:
#def tree_has_topic(topic, tree):
    

ModuleNotFoundError: No module named 'flatdict'

In [78]:
#build the tree


def get_leaves(topic, triples):
    node = {}
    node[topic] = {"children": []}
    #topic = node
    for t in triples:
        if t[0] == topic and t[2] == "child": #or t[1] == topic and t[2] == "parent":
            node[topic]["children"] += [get_leaves(t[1], triples)] 
            
            # TODO: flatten and check if key is already in 
    if len(node[topic]["children"]) == 0: # leave
        return node
    return node

In [79]:
x = "disease"
[t for t in triples if t[0] == x or t[1] == x]

ERROR! Session/line number was not unique in database. History logging moved to new session 4


[('diabetic', 'disease', 'parent'),
 ('disease', 'cardiovascular', 'child'),
 ('disease', 'myocardial', 'child'),
 ('disease', 'inflammatory', 'child'),
 ('disease', 'bowel', 'child'),
 ('disease', 'gastrointestinal', 'child'),
 ('disease', 'esophageal', 'child'),
 ('disease', 'eosinophilic', 'child'),
 ('disease', 'diseases', 'child'),
 ('disease', 'disorders', 'child'),
 ('disease', 'igg4', 'child'),
 ('disease', 'dysphagia', 'child'),
 ('disease', 'skin', 'child')]

In [ ]:
trees = {}
for triple in triples:
    #every entity first get's it's children attached before attaching to parent
    
    if not trees.get(triple[0]):
        trees[triple[0]] = get_leaves(triple[0], triples)

trees

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-78-c8fd6c62d98b>", line 6, in <module>
    trees[triple[0]] = get_leaves(triple[0], triples)
  File "<ipython-input-55-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  File "<ipython-input-55-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  File "<ipython-input-55-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  [Previous line repeated 2954 more times]
  File "<ipython-input-55-29089f7ed0f4>", line 7, in get_leaves
    if t[0] == topic and t[2] == "child" or t[1] == topic and t[2] == "parent":
RecursionError: maximum recursion depth exceeded in comparison

During handling of the above excep

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-78-c8fd6c62d98b>", line 6, in <module>
    trees[triple[0]] = get_leaves(triple[0], triples)
  File "<ipython-input-55-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  File "<ipython-input-55-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  File "<ipython-input-55-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  [Previous line repeated 2954 more times]
  File "<ipython-input-55-29089f7ed0f4>", line 7, in get_leaves
    if t[0] == topic and t[2] == "child" or t[1] == topic and t[2] == "parent":
RecursionError: maximum recursion depth exceeded in comparison

During handling of the above excep

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-78-c8fd6c62d98b>", line 6, in <module>
    trees[triple[0]] = get_leaves(triple[0], triples)
  File "<ipython-input-55-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  File "<ipython-input-55-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  File "<ipython-input-55-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  [Previous line repeated 2954 more times]
  File "<ipython-input-55-29089f7ed0f4>", line 7, in get_leaves
    if t[0] == topic and t[2] == "child" or t[1] == topic and t[2] == "parent":
RecursionError: maximum recursion depth exceeded in comparison

During handling of the above excep

In [ ]:
#get_leaves("health", triples)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-55-43fc4d19076c>", line 1, in <module>
    get_leaves("health", triples)
  File "<ipython-input-53-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  File "<ipython-input-53-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  File "<ipython-input-53-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  [Previous line repeated 2954 more times]
  File "<ipython-input-53-29089f7ed0f4>", line 7, in get_leaves
    if t[0] == topic and t[2] == "child" or t[1] == topic and t[2] == "parent":
RecursionError: maximum recursion depth exceeded in comparison

During handling of the above exception, another except

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-55-43fc4d19076c>", line 1, in <module>
    get_leaves("health", triples)
  File "<ipython-input-53-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  File "<ipython-input-53-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  File "<ipython-input-53-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  [Previous line repeated 2954 more times]
  File "<ipython-input-53-29089f7ed0f4>", line 7, in get_leaves
    if t[0] == topic and t[2] == "child" or t[1] == topic and t[2] == "parent":
RecursionError: maximum recursion depth exceeded in comparison

During handling of the above exception, another except

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-55-43fc4d19076c>", line 1, in <module>
    get_leaves("health", triples)
  File "<ipython-input-53-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  File "<ipython-input-53-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  File "<ipython-input-53-29089f7ed0f4>", line 8, in get_leaves
    node[topic]["children"] += [get_leaves(t[1], triples)]
  [Previous line repeated 2954 more times]
  File "<ipython-input-53-29089f7ed0f4>", line 7, in get_leaves
    if t[0] == topic and t[2] == "child" or t[1] == topic and t[2] == "parent":
RecursionError: maximum recursion depth exceeded in comparison

During handling of the above exception, another except

In [23]:

for topic, sources in td.items():
    #every combination gets overlaps
    

disease
inflammatory
gastrointestinal
esophageal
eosinophilic
diseases
disorders
depression
mental
health
suicide
stroke
disability
parkinson
pd
diabetes
diabetic
cardiovascular
myocardial
cancer
skin
infection
tuberculosis
rice
nursing
students
eating
children
overweight
adherence
obesity
medication
